<a href="https://colab.research.google.com/github/lakshmansamvith/Digit-Recognizer/blob/master/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow 

url1 = 'https://raw.githubusercontent.com/lakshmansamvith/Digit-Recognizer/master/train.csv'
url2 = 'https://raw.githubusercontent.com/lakshmansamvith/Digit-Recognizer/master/test.csv'
training_data_frame = pd.read_csv(url1)
testing_data_frame = pd.read_csv(url2)
training_data_frame

#Checking for Null Values in Columns 

In [0]:
testing_data_frame.isnull().any()
sum(testing_data_frame.isnull().sum())
training_data_frame.isnull().any()
sum(training_data_frame.isnull().sum())
sns.countplot(training_data_frame['label'])

#Preprocessing Data

In [0]:
y = training_data_frame[['label']].values.reshape(len(training_data_frame), )
X = pd.DataFrame(training_data_frame.drop(labels = ["label"],axis = 1))
print(type(X))

#Data Normalization

In [0]:
X = X/255.0
testing_data_frame = testing_data_frame/255.0
print(type(X), type(y))

#Reshaping Data and Splitting Data 

In [0]:
from sklearn.model_selection import train_test_split

X = X.values.reshape(-1, 28, 28, 1)
y.reshape(len(X),)
testing_data_frame = testing_data_frame.values.reshape(-1, 28, 28, 1)
train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#Print all the shapes(Reference)

In [0]:
type(X), type(y), X.shape, X_train.shape, y.shape , y_train.shape


#Creating a Convolution Network 

In [0]:
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Dropout, Flatten
from tensorflow.keras.models import Sequential 

model = Sequential([Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)), 
                   MaxPooling2D(2, 2),
                   Dropout(0.2),
                   Conv2D(32, (3,3), activation='relu'),
                   MaxPooling2D(2, 2),
                   Dropout(0.2), 
                   Conv2D(64, (3,3), activation='relu'),
                   MaxPooling2D(2, 2),
                   Dropout(0.2), 
                   Flatten(),
                   Dense(512, activation='relu' ),
                   Dropout(0.4),
                   Dense(5, activation='softmax') 
                   ])

#Data Augmentation 

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
training_datagen = ImageDataGenerator(shear_range=0.2, 
                                         rotation_range=50, 
                                         zoom_range=0.1, 
                                         rescale=1/255, 
                                         height_shift_range=40, 
                                         width_shift_range=40
                                           )
validation_datagen = ImageDataGenerator(rescale=1/255.0)

training_generator= training_datagen.flow(X_train, y_train, batch_size=10)
validation_generator = validation_datagen.flow(X_test, y_test, batch_size=10) 


#CallBack function 

In [0]:
class myCallback(tensorflow.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('val_acc')>0.90):
                print("\nReached 90% accuracy so cancelling training!")
                self.model.stop_training = True



#Model Compilation and Model Fitting 

In [0]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(), metrics=['accuracy'], loss='categorical_crossentropy')
image1 = plt.imshow(X_train[10][:,:, 0])

callback = myCallback()

history = model.fit_generator(training_generator, 
                               epochs = 30, 
                               validation_data = validation_generator, 
                               verbose = 2,
                               steps_per_epoch=(X_train.shape[0])//10, 
                              callbacks=[callback])